<a href="https://colab.research.google.com/github/neworderby/SQL_Practice/blob/main/SQL_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Заливка данных и импорт библиотек

In [15]:
import sqlite3

In [16]:
sqlite3.sqlite_version

'3.37.2'

In [17]:
con = sqlite3.connect('db')

In [18]:
import pandas as pd
import numpy as np

In [19]:
import datetime as dt

In [20]:
def select(sql):
  return pd.read_sql(sql,con)

https://docs.google.com/spreadsheets/d/1g1ecxs8SqoT-9H_FGgZGtYKAIBd0SL35/edit?usp=sharing&ouid=113443512072504582473&rtpof=true&sd=true

In [25]:
! gdown --id 1g1ecxs8SqoT-9H_FGgZGtYKAIBd0SL35

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1g1ecxs8SqoT-9H_FGgZGtYKAIBd0SL35
To: /content/sales.xlsx
100% 11.0k/11.0k [00:00<00:00, 31.1MB/s]


In [26]:
sales = pd.read_excel('/content/sales.xlsx')

### Преобразование в SQL

In [31]:
sales.to_sql('sales', con, index=False, if_exists = 'replace')

72

# Оконные функции для аналитики

#### Задачи

##### Описание таблицы

In [34]:
sql = '''
select * from sales
where year = 2020
'''
select(sql)

,year,month,quarter,plan,price,quantity,revenue
0,2020,1,1,silver,60,450,27000
1,2020,1,1,gold,240,60,14400
2,2020,1,1,platinum,600,12,7200
3,2020,2,1,silver,60,1020,61200
4,2020,2,1,gold,240,121,29040
5,2020,2,1,platinum,600,22,13200
6,2020,3,1,silver,60,700,42000
7,2020,3,1,gold,240,130,31200
8,2020,3,1,platinum,600,28,16800
9,2020,4,2,silver,60,700,42000


##### Выручка по тарифу gold за 2020

In [ ]:
sql = '''
with data as (
  select year, month, revenue
  from sales
  where year = 2020 and plan = 'gold'
)

select
  *, lag(revenue, 1) over w as prev,
  round(revenue*100.0/lag(revenue, 1) over w) as perc
from data
window w as (order by month)
order by month
'''
select(sql)

,year,month,revenue,prev,perc
0,2020,1,14400,NaN,NaN
1,2020,2,29040,14400.0,202.0
2,2020,3,31200,29040.0,107.0
3,2020,4,43200,31200.0,138.0
4,2020,5,34560,43200.0,80.0
5,2020,6,44880,34560.0,130.0
6,2020,7,40320,44880.0,90.0
7,2020,8,28800,40320.0,71.0
8,2020,9,28800,28800.0,100.0
9,2020,10,36000,28800.0,125.0


##### Выручка по тарифам за 1 квартал 2020

In [ ]:
sql = '''
select plan, year, month, revenue, sum(revenue) over w as total
from sales
where quarter = 1 and year = 2020
window w as (partition by plan order by month)
order by plan, month
'''
select(sql)

,plan,year,month,revenue,total
0,gold,2020,1,14400,14400
1,gold,2020,2,29040,43440
2,gold,2020,3,31200,74640
3,platinum,2020,1,7200,7200
4,platinum,2020,2,13200,20400
5,platinum,2020,3,16800,37200
6,silver,2020,1,27000,27000
7,silver,2020,2,61200,88200
8,silver,2020,3,42000,130200


##### Скользящее среднее по тарифу platinum за 2020

In [ ]:
sql = '''
select year, month, revenue, round(avg(revenue) over w) as avg3m
from sales
where plan = 'platinum' and year = 2020
window w as (order by month
rows between 1 preceding and 1 following)
order by month
'''
select(sql)

,year,month,revenue,avg3m
0,2020,1,7200,10200.0
1,2020,2,13200,12400.0
2,2020,3,16800,18400.0
3,2020,4,25200,22000.0
4,2020,5,24000,27200.0
5,2020,6,32400,28400.0
6,2020,7,28800,24800.0
7,2020,8,13200,18600.0
8,2020,9,13800,15000.0
9,2020,10,18000,22600.0


##### Сравнение с декабрем

In [ ]:
sql = '''
select year, month, revenue, last_value(revenue) over w as december,
round(revenue*100.0/last_value(revenue) over w) as perc
from sales
where plan = 'silver'
window w as (partition by year order by month
 rows between unbounded preceding and unbounded following)
order by year, month
'''
select(sql)

,year,month,revenue,december,perc
0,2019,1,12000,26400,45.0
1,2019,2,39600,26400,150.0
2,2019,3,24000,26400,91.0
3,2019,4,18000,26400,68.0
4,2019,5,26400,26400,100.0
5,2019,6,32400,26400,123.0
6,2019,7,26400,26400,100.0
7,2019,8,26400,26400,100.0
8,2019,9,15000,26400,57.0
9,2019,10,25200,26400,95.0


##### Вклад тарифов

In [ ]:
sql = '''
with data as (
select year, plan, sum(revenue) as revenue
from sales
group by year, plan
order by year)
select *, sum(revenue) over w as total, round(revenue*100/sum(revenue) over w) as perc
from data
window w as (partition by year)
order by year, plan
'''
select(sql)

,year,plan,revenue,total,perc
0,2019,gold,252960,722460,35.0
1,2019,platinum,168000,722460,23.0
2,2019,silver,301500,722460,41.0
3,2020,gold,411840,1244940,33.0
4,2020,platinum,249600,1244940,20.0
5,2020,silver,583500,1244940,46.0


##### Высокая, средняя и низкая выручка

In [ ]:
sql = '''
with data as (
select year, month, sum(revenue) as revenue
from sales
where year = 2020
group by month
)
select *,
ntile(3) over w as tile
from data
window w as (order by revenue desc)
order by revenue desc
'''
select(sql)

,year,month,revenue,tile
0,2020,11,150540,1
1,2020,6,130080,1
2,2020,7,115920,1
3,2020,12,115800,1
4,2020,10,111000,2
5,2020,4,110400,2
6,2020,2,103440,2
7,2020,5,97560,2
8,2020,9,96600,3
9,2020,3,90000,3


##### 2020 vs 2019

In [ ]:
sql = '''
with data as (
select year, quarter, sum(revenue) as revenue
from sales
group by year, quarter
order by 1,2)

select * from (
select *, first_value(revenue) over w as prev,
round(revenue*100.0/first_value(revenue) over w) as perc
from data
window w as (
  partition by quarter
  rows between unbounded preceding and unbounded following
)
order by 1,2) t
where year = 2020
'''
select(sql)

,year,quarter,revenue,prev,perc
0,2020,1,242040,155040,156.0
1,2020,2,338040,162600,208.0
2,2020,3,287520,204120,141.0
3,2020,4,377340,200700,188.0


##### Рейтинг месяцев по количеству продаж в 2020

In [ ]:
sql = '''
with data as (
select year, month, sum(silver) as silver, sum(gold) as gold, sum(platinum) as platinum from (
select year, month,  case when plan = 'silver' then sum(quantity) else null end as silver,
case when plan = 'gold' then sum(quantity) else null end as gold,
case when plan = 'platinum' then sum(quantity) else null end as platinum
from sales
where year = 2020
group by month, plan
order by 1,2) t
group by 1,2
order by 1,2)

select year, month,
rank() over (order by silver desc) as silver,
rank() over (order by gold desc) as gold,
rank() over (order by platinum desc) as platinum
from data
order by 1,2
'''
select(sql)

,year,month,silver,gold,platinum
0,2020,1,12,12,12
1,2020,2,3,8,10
2,2020,3,8,7,8
3,2020,4,8,3,4
4,2020,5,10,6,5
5,2020,6,6,2,2
6,2020,7,7,4,3
7,2020,8,11,9,10
8,2020,9,5,9,9
9,2020,10,4,5,7


In [ ]:
sql = '''
select year, month
, rank() over(order by sum(case when plan = 'silver' then quantity end) desc) silver
, rank() over(order by sum(case when plan = 'gold' then quantity end) desc) gold
, rank() over(order by sum(case when plan = 'platinum' then quantity end) desc) platinum
from sales
where year = 2020
group by month
order by month
'''
select(sql)

,year,month,silver,gold,platinum
0,2020,1,12,12,12
1,2020,2,3,8,10
2,2020,3,8,7,8
3,2020,4,8,3,4
4,2020,5,10,6,5
5,2020,6,6,2,2
6,2020,7,7,4,3
7,2020,8,11,9,10
8,2020,9,5,9,9
9,2020,10,4,5,7
